In [1]:
from nltk.metrics.distance import edit_distance

In [2]:
correct_words = ['gulshan e hadeed', 'gulshan e iqbal', 'defence', 'clifton', 'meher plaza', 'al murtaza heights', 'al murtaza height']

incorrect_words= ['gulshen iqbal', 'defnse', 'klifton', 'mehar plaza', 'al murteza heights']

for word in incorrect_words:
		temp = [(edit_distance(word, w),w) for w in correct_words]
		# print(sorted(temp, key = lambda val:val[0])[0][1])

		if len(temp) > 1: 
			temp = [min(temp, key=lambda t: t[0])]

		print(temp)
		# print(temp[0][1])

# print(edit_distance(incorrect_words[2], correct_words[3]), correct_words[3])

[(3, 'gulshan e iqbal')]
[(2, 'defence')]
[(1, 'clifton')]
[(1, 'meher plaza')]
[(1, 'al murtaza heights')]


main focus of address parsing - building names and uniformaty

2 parts: 
- data parsing and storing in our DB (standart uniform way)
- checking in DB, if same address fields are there so directly storing fields in db from prior knowledge

data pipeline shape to the project (proper flow plus maximum automation (function automatic calling inside another function))

organize files into folders, potiental folders (sub folders as required) : 
- code (main folder can already be code one)
- data bases/data


In [3]:
import json
import string

def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

def lowercase_text(text):
    return text.lower()

def process_json(json_data):
    processed_data = {}
    for key, value in json_data.items():
        processed_key = remove_punctuation(lowercase_text(key))
        processed_value = remove_punctuation(lowercase_text(value))
        processed_data[processed_key] = processed_value
    return processed_data

# Read the JSON file
filename = "abbreviations.json"
with open(filename) as file:
    data = json.load(file)

# Process the JSON data
processed_data = process_json(data)

# Write the processed data back to the JSON file
with open(filename, "w") as file:
    json.dump(processed_data, file, indent=4)

print(f"JSON file '{filename}' has been modified.")



JSON file 'abbreviations.json' has been modified.


length of df = 213874

In [ ]:
area = ['block', 'phase', 'scheme', 'sector']

street = ['street', 'phase', 'road', 'highway', 'khayaban']

In [ ]:
columns = ['Ticket #', 'Type' 'Unit #', 'Street Name', 'Building #', 'Building Name', 'City']

place index number with every token

Hierarchy: 

(if present because not all feilds in all addresses)

- Type --> INFERRED

- Unit #


- Building #

- Building Name 

- Street Number/Name --> KEYWORD CHECK

- Sub Area (phase, block, etc.) --> KEYWORD CHECK

- Area (eg: bath island, civil lines) --> IF AVAILABLE IN END

- Neighbourhood  --> STANDART, [-2] in address string

- City --> STANDART, [-1] in address string

Each hierarchy step check:

- pre_processing function --> grammer, punctuation, spacing, capitalization

- tokenization

- one by one tokenization feilds checks:

    - spelling check (where applicable like neighbour hoods)

    - checking if already in db

    - keyword check where to place and place

- place in DB